In [1]:
# Install prerequisites

# Install PyCocoTools needed for FasterRCNN
!pip install git+https://github.com/gautamchitnis/cocoapi.git@cocodataset-master#subdirectory=PythonAPI

# Copy useful functions from pytorch vision tools
%cp ../input/pytorch-vision-tools/references/detection/*.* .

  Cloning https://github.com/gautamchitnis/cocoapi.git (to revision cocodataset-master) to /tmp/pip-req-build-ob_xh0z9
  Running command git clone -q https://github.com/gautamchitnis/cocoapi.git /tmp/pip-req-build-ob_xh0z9
  Running command git checkout -b cocodataset-master --track origin/cocodataset-master
  Switched to a new branch 'cocodataset-master'
  Branch 'cocodataset-master' set up to track remote branch 'cocodataset-master' from 'origin'.
  Created wheel for pycocotools: filename=pycocotools-2.0-cp37-cp37m-linux_x86_64.whl size=272668 sha256=76cde0a1c4ece94243036342d72384843055e08c193c7f132121e3d0a4dd746a
  Stored in directory: /tmp/pip-ephem-wheel-cache-nmd438f5/wheels/6e/c9/59/56484d4d5ac1ab292a452b4c3870277256551505954fc4a1db
Successfully built pycocotools


In [2]:
# Import required libraries
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader, Subset
from PIL import Image
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from os import path
import torch
import torchvision.utils
import utils
from engine import train_one_epoch, evaluate
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.optim import Adam
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import pickle
import time

In [3]:
# Define a dataset class which defines how to load images,targets for training and validation
class VinBigDataset(Dataset):
    def __init__(self, img_dir, df, transforms):
        self.img_dir = img_dir
        self.df = df
        self.imgs = df["image_id"].unique()
        self.transforms = transforms
        
    def __len__(self):
        # Return the number of elements in the dataset
        return len(self.imgs)
    
    def __getitem__(self, idx):
        # The dataset iterates over each image id
        # Return the requested image,target from the dataset
        
        # Get the id of the current image
        img_id = self.imgs[idx]
        
        # Get the rows containing annotations for this image
        data_rows = self.df[self.df["image_id"] == img_id]
        boxes = data_rows[["x_min", "y_min", "x_max", "y_max"]].values
        
        # Convert into a torch.Tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        
        # Load the image
        img_path = path.join(self.img_dir, f"{img_id}.png")
        img = Image.open(img_path).convert("RGB")
        
        # Compute the area of the annotated box
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:,0])
        
        # Suppose all instances are not crowd (?)
        num_objs = len(boxes)
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)
        
        # There is only one class, so labels are all ones
        labels = torch.ones((num_objs,), dtype=torch.int64)
        
        # Define the target for this training data point
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = torch.tensor([idx])
        target["area"] = area
        target["iscrowd"] = iscrowd
        
        if self.transforms is not None:
            to_transform = {
                'image': np.array(img),
                'bboxes': target['boxes'],
            }
            transformed = self.transforms(**to_transform)
            img = transformed['image']    
            target['boxes'] = torch.tensor(transformed['bboxes'])
        
        return img, target

In [4]:
# Function to create transforms for preprocessing
def create_transforms(train):
    if train:
        return A.Compose([ 
#             A.Flip(0.5),
            A.Normalize(mean=(0, 0, 0), std=(1, 1, 1), max_pixel_value=255.0, p=1.0), 
            ToTensorV2(p=1.0)
        ])
    else:
        return A.Compose([
            A.Normalize(mean=(0, 0, 0), std=(1, 1, 1), max_pixel_value=255.0, p=1.0), 
            ToTensorV2(p=1.0)
        ])

In [5]:
# Function to create an instance of the model
def create_model():
    # The model has 14 classes
    num_classes = 14
    
    # Use resnet50 pre-trained on COCO
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
        
    # Fetch the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    
    # Replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
   
    return model

In [6]:
# Load the training data csv file
data_dir = "../input/vinbigdata1024mlp/"

df_train = pd.read_csv(f"{data_dir}/train.csv")

# Remove images that do not contain any anomolies
df_train = df_train[df_train["class_id"]!=14]

# Print out the total number of images and the total number of annotations 
print(f"{df_train['image_id'].nunique()} images")
print(f"{len(df_train.index)} annotations")

4394 images
23961 annotations


In [7]:
# Create an instance of the dataset and transformations for training
# and validation

train_img_dir = path.join(data_dir, 'train')
# train_img_dir = path.join(data_dir, 'train/train')
train_dataset = VinBigDataset(train_img_dir, df_train, create_transforms(train=True))
val_dataset = VinBigDataset(train_img_dir, df_train, create_transforms(train=False))

# Permute the dataset and split into training and validation set 9:1
torch.manual_seed(5262394)
indices = torch.randperm(len(train_dataset)).tolist()
val_set_size = len(train_dataset) // 10

train_dataset = Subset(train_dataset, indices[:-val_set_size])
val_dataset = Subset(val_dataset, indices[-val_set_size:])

# Create data loaders for the training and validation set. The collate function
# defines how to form a minibatch from the indiviaual data items. In our case we
# just want to collate them into a single list.

data_loader_train = DataLoader(\
    train_dataset, batch_size=5, shuffle=True, num_workers=4, collate_fn=utils.collate_fn)

data_loader_val = DataLoader(\
    val_dataset, batch_size=1, shuffle=False, num_workers=4, collate_fn=utils.collate_fn)

# Print the number of elements in the test and training set
print(f"{len(train_dataset)} items in the training set")
print(f"{len(val_dataset)} items in the validation set")

3955 items in the training set
439 items in the validation set


In [8]:
# Cloud Storage
from google.cloud import storage
import os
storage_client = storage.Client(project='s2117072-mlpractical')

def create_bucket(bucket_name):
 bucket = storage_client.create_bucket(bucket_name)

def upload_file(bucket_name, filename):
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(filename)
    blob.upload_from_filename(filename)
    
def download_file(bucket_name, filename):
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(filename)
    blob.download_to_filename(filename)

def upload_files(bucket_name, source_folder):
 bucket = storage_client.get_bucket(bucket_name)
 for filename in os.listdir(source_folder):
     blob = bucket.blob(filename)
     blob.upload_from_filename(source_folder + filename)

In [9]:
model = create_model()

use_pre_trained = True
if use_pre_trained:
    bucket_name = 's2117072_mlp_20210308_130504'
    os.makedirs('model_weights', exist_ok=True)
    blob_name = 'model_weights/model_weights.bin'
    download_file(bucket_name, blob_name)
    model.load_state_dict(torch.load(blob_name))

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


  0%|          | 0.00/160M [00:00<?, ?B/s]

In [10]:
# Train the model
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model.to(device)

# Hyperparameters
learning_rate = 0.001
weight_decay = 0
num_epochs = 20

params = [p for p in model.parameters() if p.requires_grad]

optimizer = Adam(params, lr=learning_rate, weight_decay=weight_decay)
lr_scheduler = CosineAnnealingLR(optimizer, T_max=num_epochs, eta_min=0.00002)

# Create a folder for saving the model weights
%mkdir model_weights
%mkdir stats

train_meters = []
coco_evals = []
t_start = time.time()

for epoch in range(num_epochs):
    # Train over the epoch
    logger = train_one_epoch(model, optimizer, data_loader_train, device, epoch, print_freq=100)
    train_meters.append(logger.meters)
    
    # Evaluate on the validation set
    coco_eval = evaluate(model, data_loader_val, device)
    coco_evals.append(coco_eval.coco_eval)
    
    # Update the learning rate
    lr_scheduler.step()

t_end = time.time()
print(f"Training took {t_end - t_start}")
      
# Save the model weights
torch.save(model.state_dict(), f"model_weights/model_weights.bin")
    
# Save the stats
pickle.dump(train_meters, open(f"stats/train_stats.pkl", 'wb'))
pickle.dump(coco_evals, open(f"stats/val_stats.pkl", 'wb'))
    

mkdir: cannot create directory ‘model_weights’: File exists


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(Tru

Epoch: [0]  [  0/791]  eta: 0:37:12  lr: 0.000002  loss: 0.4973 (0.4973)  loss_classifier: 0.1297 (0.1297)  loss_box_reg: 0.3316 (0.3316)  loss_objectness: 0.0092 (0.0092)  loss_rpn_box_reg: 0.0268 (0.0268)  time: 2.8219  data: 1.0362  max mem: 3745
Epoch: [0]  [100/791]  eta: 0:07:26  lr: 0.000129  loss: 0.3413 (0.3620)  loss_classifier: 0.1019 (0.1003)  loss_box_reg: 0.2036 (0.2228)  loss_objectness: 0.0097 (0.0108)  loss_rpn_box_reg: 0.0251 (0.0281)  time: 0.6207  data: 0.0251  max mem: 4170
Epoch: [0]  [200/791]  eta: 0:06:15  lr: 0.000255  loss: 0.3565 (0.3773)  loss_classifier: 0.0897 (0.1057)  loss_box_reg: 0.2223 (0.2318)  loss_objectness: 0.0112 (0.0114)  loss_rpn_box_reg: 0.0202 (0.0284)  time: 0.6238  data: 0.0253  max mem: 4170
Epoch: [0]  [300/791]  eta: 0:05:10  lr: 0.000382  loss: 0.4853 (0.3902)  loss_classifier: 0.1484 (0.1119)  loss_box_reg: 0.2678 (0.2376)  loss_objectness: 0.0187 (0.0126)  loss_rpn_box_reg: 0.0194 (0.0280)  time: 0.6224  data: 0.0241  max mem: 4170


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


creating index...
index created!
Test:  [  0/439]  eta: 0:03:24  model_time: 0.1671 (0.1671)  evaluator_time: 0.0336 (0.0336)  time: 0.4655  data: 0.2519  max mem: 4170
Test:  [100/439]  eta: 0:00:38  model_time: 0.0624 (0.0668)  evaluator_time: 0.0279 (0.0328)  time: 0.1149  data: 0.0057  max mem: 4170
Test:  [200/439]  eta: 0:00:26  model_time: 0.0622 (0.0652)  evaluator_time: 0.0221 (0.0332)  time: 0.1087  data: 0.0055  max mem: 4170
Test:  [300/439]  eta: 0:00:15  model_time: 0.0628 (0.0644)  evaluator_time: 0.0366 (0.0334)  time: 0.1179  data: 0.0059  max mem: 4170
Test:  [400/439]  eta: 0:00:04  model_time: 0.0626 (0.0645)  evaluator_time: 0.0292 (0.0345)  time: 0.1159  data: 0.0060  max mem: 4170
Test:  [438/439]  eta: 0:00:00  model_time: 0.0620 (0.0642)  evaluator_time: 0.0200 (0.0341)  time: 0.0986  data: 0.0050  max mem: 4170
Test: Total time: 0:00:47 (0.1088 s / it)
Averaged stats: model_time: 0.0620 (0.0642)  evaluator_time: 0.0200 (0.0341)
Accumulating evaluation results.

In [11]:
%%bash 
zip -r -q stats.zip stats/*.*

In [12]:
from datetime import datetime
cur_time = datetime.now().strftime('%Y%m%d_%H%M%S')
bucket_name = f's2117072_mlp_{cur_time}'
create_bucket(bucket_name)

In [13]:
upload_file(bucket_name, 'stats.zip')
upload_file(bucket_name, f"model_weights/model_weights.bin")